In [167]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
import pandas as pd

url = "https://play.google.com/store/apps/details?id=com.kakao.talk&hl=ko&gl=US"
driver = webdriver.Chrome()
driver.get(url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

spread_review = driver.find_element(by=By.XPATH, value = "/html/body/c-wiz[2]/div/div/div[2]/div[2]/div/div[1]/div[1]/c-wiz[4]/section/div/div[2]/div[5]/div/div/button")
isTrue = spread_review.is_displayed()
if isTrue :
    driver.execute_script("arguments[0].click();", spread_review)
    time.sleep(1.5)

all_reviews = driver.find_element(by=By.XPATH, value ='//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]')

def scroll_down(n):
    for i in range(n):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', all_reviews)
        time.sleep(1)

scoll_pages = 10
scroll_down(scoll_pages)
    
data = pd.DataFrame(data=[], columns=['date','review','rating'])
#날짜, 리뷰, 별점 수집

dates = driver.find_elements(by=By.XPATH, value = '//span[@class="bp9Aid"]')
reviews=driver.find_elements(by=By.XPATH, value = '//div[@class="h3YV2d"]')
stargrades = driver.find_elements(by=By.XPATH, value = '//div[@class="iXRFPc"]')

for i in range(len(reviews)):
    tmp = []
    tmp.append(dates[i].text)
    tmp.append(reviews[i].text)
    tmp.append(stargrades[i].get_attribute('aria-label'))
    
        
    tmp = pd.DataFrame(data=[tmp], columns = data.columns)
    data = pd.concat([data,tmp])

data.reset_index(inplace=True, drop=True)
data = data.drop(data.index[:3])
tmp = data.copy()
tmp['rating'] = tmp['rating'].apply(lambda x : x[5:])
m = re.compile('[0-9][\.0-9]*') #정규표현식
tmp['rating'] = tmp['rating'].apply(lambda x : m.findall(x)[0])

tmp['date'] = pd.to_datetime(tmp['date'], format='%Y년 %m월 %d일')
tmp['date'] = tmp['date'].dt.strftime('%Y-%m-%d')
tmp.to_csv('kakaotalk_review.csv', encoding='utf-8-sig')
tmp

,date,review,rating
3,2023-10-22,톡서랍플러스 이용자입니다. 폰으로 주고받은 대화들을 pc 에서 검색할때 해당 기간을...,4
4,2023-11-17,보이스톡이 안돼요. 상대방이 보이스톡을 걸면 상단바랑 화면에 알림이 뜨는데 통화도 ...,2
5,2023-11-13,"매우 편리 합니다 문자를 보낼 수 있고 쇼핑,이모티콘 등 여러가지 카테고리를 할 수...",4
6,2023-11-18,"전화와 문자를 무료로 할 수 있는 건 진짜 좋은 시스템인 것 같지만, 몇가지 단점이...",4
7,2023-09-17,다른건 다 좋습니다. 다만 최근 업데이트 후 보이스톡이나 페이스톡을 걸면 앱이 강제...,3
...,...,...,...
78,2023-11-23,"보이스톡 고쳐주세요, 보이스톡하기 눌렀는데 화면이 안보이고, 끊기도 안눌리고, 상대...",1
79,2023-10-09,ㅋㅋㅋㅋㅋㅋ 한번 다른 기기로 로그인하고 나니깐 제가 사용하는 기기에 보이스톡이 버...,1
80,2023-09-23,"카카오톡 잘 쓰고 있습니다, 다만 여러 전세계 사람들이 많이 써서 이것 만 고쳐주시...",4
81,2023-10-09,이용자 보호조치좀 적당히 거세요 무슨 오픈채팅방 3개 들어갔다고 3번째부터 막아버리...,1


In [32]:
import google_play_scraper as gps

country = 'us'
lang = 'ko'
id = 'com.simplywerx.compass3d'

result_a  = gps.reviews_all(
    id,
    sleep_milliseconds=0,# defaults to 0
    lang=lang,  # defaults to 'en'
    country=country,  # defaults to 'us'
    sort=gps.Sort.NEWEST,  # defaults to Sort.MOST_RELEVANT
    count = 3,
    filter_score_with=None  # defaults to None(means all score)
)
result_a
# for item in result_a:
#     print(str(item['at'])+'\t'+item['userName']+'\t'+str(item['score'])+'\t'+item['content'])

[{'reviewId': '222da2cc-5442-4333-aef3-56677dfd793b',
  'userName': '하두',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjVeOpDeKXuDp7aB0PDxIIGxqgELONTXuiUohVnAZ7boF0Y',
  'content': '제 폰은 센서가 없어서 어떡해야할지 모르겠네요..',
  'score': 3,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': None,
  'at': datetime.datetime(2023, 11, 21, 21, 17, 28),
  'replyContent': None,
  'repliedAt': None,
  'appVersion': None},
 {'reviewId': 'ca8c26d9-f818-40ee-a0de-3b5ec937f24d',
  'userName': 'yun행복하세요',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjWxmcGhrgoEw0kvHUBl36zbkGewpybwjb87B52Lgkqx4bw',
  'content': '동작 잘됩니다!',
  'score': 5,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': '3.6.0',
  'at': datetime.datetime(2023, 10, 5, 19, 38, 30),
  'replyContent': None,
  'repliedAt': None,
  'appVersion': '3.6.0'},
 {'reviewId': 'ee3734be-2865-4077-b371-aba58a342817',
  'userName': '최영철',
  'userImage': 'https://play-lh.googleusercontent.com/a/ACg8ocJSrdeEnWnQiERShsIxmHGWL2Nk0PrFe2n1f

In [67]:
from google_play_scraper import Sort, reviews
from datetime import datetime

result, _ = reviews(
    'com.kakao.talk',
    lang='ko',
    country='us',
    sort=Sort.MOST_RELEVANT,
    count=3000000,
    filter_score_with=None
)

df = pd.DataFrame(result)
kakaotalk = df[['at','score','thumbsUpCount','content']]
kakaotalk['at'] = kakaotalk['at'].dt.strftime('%Y-%m-%d')
kakaotalk

# for item in result:
#     formatted_datetime = item['at'].strftime('%Y-%m-%d %H:%M:%S')
#     time_only = formatted_datetime.split(' ')[0]  # 시간 부분만 추출
#     print(time_only +'\t'+str(item['score'])+'\t' + str(item['thumbsUpCount']) + '\t' +item['content'])

C:\Users\COMPUTER\AppData\Local\Temp/ipykernel_22804/1202785815.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kakaotalk['at'] = kakaotalk['at'].dt.strftime('%Y-%m-%d')


,at,score,thumbsUpCount,content
0,2023-10-22,4,36,톡서랍플러스 이용자입니다. 폰으로 주고받은 대화들을 pc 에서 검색할때 해당 기간을...
1,2023-11-18,2,1022,보이스톡이 안돼요. 상대방이 보이스톡을 걸면 상단바랑 화면에 알림이 뜨는데 통화도 ...
2,2023-11-13,4,1400,"매우 편리 합니다 문자를 보낼 수 있고 쇼핑,이모티콘 등 여러가지 카테고리를 할 수..."
3,2023-11-18,4,695,"전화와 문자를 무료로 할 수 있는 건 진짜 좋은 시스템인 것 같지만, 몇가지 단점이..."
4,2023-09-17,3,2482,다른건 다 좋습니다. 다만 최근 업데이트 후 보이스톡이나 페이스톡을 걸면 앱이 강제...
...,...,...,...,...
107853,2018-12-06,3,1,선물하기 기능이 안되요.
107854,2018-12-09,4,0,그저좋아유
107855,2021-11-12,1,0,쓸만하0
107856,2018-12-10,5,0,빠른개선 좋네요 굿


In [68]:
kakaotalk

,at,score,thumbsUpCount,content
0,2023-10-22,4,36,톡서랍플러스 이용자입니다. 폰으로 주고받은 대화들을 pc 에서 검색할때 해당 기간을...
1,2023-11-18,2,1022,보이스톡이 안돼요. 상대방이 보이스톡을 걸면 상단바랑 화면에 알림이 뜨는데 통화도 ...
2,2023-11-13,4,1400,"매우 편리 합니다 문자를 보낼 수 있고 쇼핑,이모티콘 등 여러가지 카테고리를 할 수..."
3,2023-11-18,4,695,"전화와 문자를 무료로 할 수 있는 건 진짜 좋은 시스템인 것 같지만, 몇가지 단점이..."
4,2023-09-17,3,2482,다른건 다 좋습니다. 다만 최근 업데이트 후 보이스톡이나 페이스톡을 걸면 앱이 강제...
...,...,...,...,...
107853,2018-12-06,3,1,선물하기 기능이 안되요.
107854,2018-12-09,4,0,그저좋아유
107855,2021-11-12,1,0,쓸만하0
107856,2018-12-10,5,0,빠른개선 좋네요 굿


In [66]:
kakaotalk.to_csv('kakaotalk_review.csv', encoding='utf-8-sig')

In [28]:
import google_play_scraper as gps

country = 'us'
lang = 'ko'
id = 'com.kakao.talk'

result_a  = gps.reviews_all(
    id,
    sleep_milliseconds=3000,  # defaults to 0
    count = 5,
    lang=lang,  # defaults to 'en'
    country=country,  # defaults to 'us'
    sort=gps.Sort.NEWEST,  # defaults to Sort.MOST_RELEVANT
    filter_score_with=None  # defaults to None(means all score)
)

for item in result_a:
    print(str(item['at'])+'\t'+item['userName']+'\t'+str(item['score'])+'\t'+item['content'])

KeyboardInterrupt: 

In [11]:
result

[{'reviewId': 'b322957b-b240-4d76-bc7f-5ce26b8795bf',
  'userName': 'Feisty Boi',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjVmhzttMFl-HV0a2W_3FTJ_v4whlUyGXbb1MtssvP9iQrw',
  'content': 'Very calming app.',
  'score': 5,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': '1.65.1',
  'at': datetime.datetime(2023, 11, 27, 7, 15, 50),
  'replyContent': None,
  'repliedAt': None,
  'appVersion': '1.65.1'},
 {'reviewId': '6a61dc2f-c247-40df-acaa-0d9dbca008f3',
  'userName': 'Franklin Soros',
  'userImage': 'https://play-lh.googleusercontent.com/a-/ALV-UjXd7i63dXKKCEuinlvPIowEhJJN4a_-4dzwIXjuk1IXlQ',
  'content': 'Pretty relaxing and warm scenery！',
  'score': 5,
  'thumbsUpCount': 0,
  'reviewCreatedVersion': '1.65.1',
  'at': datetime.datetime(2023, 11, 27, 1, 4, 35),
  'replyContent': None,
  'repliedAt': None,
  'appVersion': '1.65.1'},
 {'reviewId': '9ca48dde-0921-4a7b-a6ef-a819ca6eb803',
  'userName': 'Kidnappingfreezers',
  'userImage': 'https://play-lh.googleusercont

In [33]:
from google_play_scraper import Sort, reviews

result, continuation_token = reviews(
    'com.fantome.penguinisle',
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    count=3, # defaults to 100
    filter_score_with=5 # defaults to None(means all score)
)

# If you pass `continuation_token` as an argument to the reviews function at this point,
# it will crawl the items after 3 review items.

result, _ = reviews(
    'com.fantome.penguinisle',
    continuation_token=continuation_token # defaults to None(load from the beginning)
)

In [34]:
result

[{'reviewId': 'ba2c136c-b800-47f3-857c-8beb6ded51bf',
  'userName': 'A Google user',
  'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nuSijCz7sc3a277R67g',
  'content': "PERFECT. I can't say enough. I really love everything about it. I don't mind the ads at all cuz you can choose to watch them or not, when you want to. They aren't automatic. Gameplay is super simple & very relaxing. Easy to understand & learn very quickly. The graphics are adorable, music is peaceful. The value of what you get for watching ads, spending coins, or spending gems is just right, easy to earn, lots of little features that make it great. Customer service is very quick/excellent!",
  'score': 5,
  'thumbsUpCount': 117,
  'reviewCreatedVersion': '1.17',
  'at': datetime.datetime(2020, 3, 25, 14, 32, 59),
  'replyContent': None,
  'repliedAt': None,
  'appVersion': '1.17'},
 {'reviewId': '9672bfaa-3d7f-4e6e-a005-5f71e173a71d',
  'userName': 'Ha